In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from scipy import stats
from sklearn.preprocessing import PowerTransformer, StandardScaler
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import time
import tqdm.notebook as tq

import lightgbm as lgb
from scipy.stats import loguniform
from random import randrange, uniform

In [2]:
customer_account = pd.read_csv("account.csv")
customer_df = pd.read_csv("customer.csv")
category_df = pd.read_csv("kategoriler.csv")
product_df = pd.read_csv("groups.csv")
train_df = pd.read_csv("train.csv")
header_df = pd.read_csv("header.csv")
sale_df = pd.read_csv("sale.csv")
test_df = pd.read_csv("test.csv")

In [6]:
def create_sale_df(product_df, sale_df):
    
    product_df, sale_df = product_df.copy(), sale_df.copy()
    
    cat_level_to_number_df = product_df.groupby(['category_level_1', 'category_level_2', 'category_level_3', 'category_level_4']).first()
    
    my_sale_df = pd.merge(sale_df, cat_level_to_number_df, on=['category_level_1', 'category_level_2', 'category_level_3', 'category_level_4'])
    my_sale_df = my_sale_df.drop(['category_level_1', 'category_level_2', 'category_level_3', 'category_level_4'], axis=1)
    my_sale_df = my_sale_df.fillna(0)
    
    my_sale_df = abs(my_sale_df)
    
    return my_sale_df

In [7]:
new_sale_df = create_sale_df(product_df, sale_df)
new_sale_df.head()

,basketid,amount,quantity,discount_type_1,discount_type_2,discount_type_3,category_number
0,20120706070177471,3.5,1.0,0.000000,0.0,0.0,9057
1,20120703120198950,10.9,1.0,0.000000,0.0,0.0,9057
2,20121704800208844,10.9,1.0,1.659883,0.0,0.0,9057
3,20121021890066582,4.9,1.0,0.000000,0.0,0.0,9057
4,20122422590012527,4.9,1.0,0.000000,0.0,0.0,9057


In [9]:
musteri_basket_df = pd.merge(header_df, customer_account, on="cardnumber", how="left").drop(["cardnumber", "date_of_transaction"], axis=1)
musteri_basket_df.head()

,basketid,is_sanal,individualnumber
0,20120163990041163,0,68318409
1,20120163990016137,0,93939734
2,20120160080017338,0,58513765
3,20120163570105022,0,93345438
4,20120162340016645,0,98388774


In [11]:
sum_basket_df = new_sale_df.drop(["category_number"], axis=1).groupby("basketid").sum().reset_index()

sum_musteri_df = pd.merge(musteri_basket_df, sum_basket_df, on="basketid").drop(["basketid"], axis=1)
sum_musteri_df = sum_musteri_df.groupby(["individualnumber"]).sum().reset_index()
sum_musteri_df.head()

,individualnumber,is_sanal,amount,quantity,discount_type_1,discount_type_2,discount_type_3
0,30690,0,5918.87,766.763,404.291503,262.54,439.58
1,294867,8,7292.49,307.340,924.600579,590.75,649.96
2,322878,10,9795.15,688.698,955.474064,650.89,315.22
3,356411,21,5813.29,552.847,173.594715,119.22,361.27
4,374368,14,8087.15,702.311,780.500682,384.38,630.72


In [13]:
sum_musteri_categ_df = pd.merge(new_sale_df, musteri_basket_df, on="basketid", how="left").drop(["basketid"], axis=1)
sum_musteri_categ_df = sum_musteri_categ_df.groupby(["individualnumber", "category_number"]).sum().reset_index()
change_col_dict = {"amount":"amount_category", "quantity":"quantity_category", 
                   "discount_type_1":"discount_type_1_category", "discount_type_2":"discount_type_2_category", 
                   "discount_type_3":"discount_type_3_category", "is_sanal":"is_sanal_category"}
sum_musteri_categ_df = sum_musteri_categ_df.rename(columns=change_col_dict)
sum_musteri_categ_df.head()

,individualnumber,category_number,amount_category,quantity_category,discount_type_1_category,discount_type_2_category,discount_type_3_category,is_sanal_category
0,30690,9000,263.95,39.0,24.967289,0.00,0.0,0
1,30690,9001,508.50,28.0,50.360283,22.65,0.0,0
2,30690,9003,10.35,3.0,0.543257,0.00,0.0,0
3,30690,9004,368.10,7.0,24.840286,6.60,0.0,0
4,30690,9006,338.10,12.0,51.476818,24.00,0.0,0


In [15]:
def create_customer_vector(customer_df_):
    
    customer_df_ = customer_df_.copy()
    
    customer_df_["gender"] = customer_df["gender"].map({"K":1, "E":0})
    # city_code -> %23 missing data & not str, float
    
    customer_df_["dateofbirth"] = customer_df_["dateofbirth"].map(lambda x: 2022-x)
    customer_df_.loc[(customer_df_["dateofbirth"] < 18) | (customer_df_["dateofbirth"] > 80), "dateofbirth"] = customer_df_["dateofbirth"].median()
    customer_df_["dateofbirth"] = customer_df_["dateofbirth"].fillna(customer_df_["dateofbirth"].median())
    
    return customer_df_

In [16]:
customer_vector = create_customer_vector(customer_df)
customer_vector.head()

,individualnumber,gender,city_code,dateofbirth
0,94212124,1.0,NaN,76.0
1,96387515,0.0,34.0,48.0
2,95040383,0.0,80.0,42.0
3,94694434,0.0,NaN,52.0
4,47648671,0.0,35.0,59.0


In [17]:
def change_pivot_col_names(vector): 
    str_column_names = []
    for a, b, c, d, e, f in vector.columns.values:
        a, b, c, d, e, f = str(a), str(b), str(c), str(d), str(e), str(f)
        if (b, c, d, e, f) == ("0", "0", "0", "0", "1"):
            ww = vector.columns.names[5]
        elif (b, c, d, e, f) == ("0", "0", "0", "1", "0"):
            ww = vector.columns.names[4]
        elif (b, c, d, e, f) == ("0", "0", "1", "0", "0"):
            ww = vector.columns.names[3]
        elif (b, c, d, e, f) == ("0", "1", "0", "0", "0"):
            ww = vector.columns.names[2]
        elif (b, c, d, e, f) == ("1", "0", "0", "0", "0"):
            ww = vector.columns.names[1]
        str_column_names.append([a, ww])

    return ['_'.join(col).strip() for col in str_column_names]

In [19]:
def create_card_vector(header_df, sale_df, product_df, category_df):
    
    # merge and drop part
    category_dict_df = product_df.groupby(["category_level_1", "category_level_2", "category_level_3", "category_level_4"]).first().reset_index()
    category_dict_df = pd.merge(category_dict_df, category_df, on="category_number", how="left")
    basked_id_vector = pd.merge(sale_df, category_dict_df, on=["category_level_1", "category_level_2", "category_level_3", "category_level_4"], how="left")
    basked_id_vector["genel_kategori"] = basked_id_vector["genel_kategori"].fillna("diger")
    basked_id_vector = basked_id_vector.drop(["category_number"], axis=1)
    basked_id_vector = basked_id_vector.drop(["category_level_1", "category_level_2", "category_level_3", "category_level_4"], axis=1)
    
    merge_df = pd.merge(header_df, basked_id_vector, on="basketid")
    how_many_basket_dict = sale_df["basketid"].value_counts().to_dict()
    merge_df["how_many_basket"] = merge_df["basketid"].map(how_many_basket_dict)
    merge_df = merge_df.drop(["basketid"], axis=1)
    
    one_hot = pd.get_dummies(merge_df["genel_kategori"])
    merge_df = merge_df.drop("genel_kategori", axis = 1)
    merge_df = merge_df.join(one_hot)
    
    change_col_name_dict = {"diger":"diger_1", "gida":"gida_1", "hijyen":"hijyen_1", "icecek":"icecek_1", "kisisel_bakim":"kisisel_bakim_1"}
    merge_df = merge_df.rename(columns=change_col_name_dict)
    
    merge_df_all_sum = merge_df.copy()
    all_sum_df = merge_df_all_sum.groupby(['cardnumber']).sum(numeric_only=True)
    all_sum_df["sum_basket"] = all_sum_df.loc[:, ["diger_1", "gida_1", "hijyen_1", "icecek_1", "kisisel_bakim_1"]].sum(axis=1)
    all_sum_df["sum_discount"] = all_sum_df.loc[:, ["discount_type_1", "discount_type_2", "discount_type_3"]].sum(axis=1)

    merge_df = merge_df.groupby(["cardnumber", "diger_1", "gida_1", "hijyen_1", "icecek_1", "kisisel_bakim_1"]).sum(numeric_only=True)
    merge_df = merge_df.reset_index().pivot_table(index='cardnumber', columns=["diger_1", "gida_1", "hijyen_1", "icecek_1", "kisisel_bakim_1"], fill_value=0) # crush dates
    
    # change pivot table column names:
    new_col_names = change_pivot_col_names(merge_df)
    merge_df.columns = new_col_names
    
    # concat
    merge_df = pd.concat([merge_df, all_sum_df], axis=1)
    
    return merge_df

In [20]:
card_vector = create_card_vector(header_df, sale_df, product_df, category_df)
card_vector.head()

,amount_kisisel_bakim_1,amount_icecek_1,amount_hijyen_1,amount_gida_1,amount_diger_1,discount_type_1_kisisel_bakim_1,discount_type_1_icecek_1,discount_type_1_hijyen_1,discount_type_1_gida_1,discount_type_1_diger_1,...,discount_type_2,discount_type_3,how_many_basket,diger_1,gida_1,hijyen_1,icecek_1,kisisel_bakim_1,sum_basket,sum_discount
cardnumber,,,,,,,,,,,,,,,,,,,,,
5142451372636,0.00,0.00,0.00,133.33,14.15,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,3.99,15,3,4,0,0,0,7,3.990000
5202438377398,0.00,395.69,534.10,1306.86,417.34,0.000000,36.459130,38.086107,39.523066,16.828691,...,244.48,106.31,493,13,47,8,15,0,83,481.686995
5272406370386,0.00,0.00,0.00,9.95,16.00,0.000000,0.000000,0.000000,0.500000,0.000000,...,0.00,0.00,4,1,1,0,0,0,2,0.500000
5742492372647,43.80,1410.05,793.16,2662.01,448.47,0.000000,48.303099,18.106476,169.886459,21.064933,...,176.72,320.76,2615,26,118,21,80,2,247,754.840967
5832490253097,43.51,95.00,203.74,866.13,806.17,22.707125,0.000000,10.160607,85.839413,113.172128,...,152.50,444.16,490,32,47,9,4,2,94,828.539273


In [22]:
all_cust_df = pd.merge(card_vector.reset_index(), customer_account, on="cardnumber", how="left").drop(["cardnumber"], axis=1)
all_cust_df.head()

,amount_kisisel_bakim_1,amount_icecek_1,amount_hijyen_1,amount_gida_1,amount_diger_1,discount_type_1_kisisel_bakim_1,discount_type_1_icecek_1,discount_type_1_hijyen_1,discount_type_1_gida_1,discount_type_1_diger_1,...,discount_type_3,how_many_basket,diger_1,gida_1,hijyen_1,icecek_1,kisisel_bakim_1,sum_basket,sum_discount,individualnumber
0,0.00,0.00,0.00,133.33,14.15,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.99,15,3,4,0,0,0,7,3.990000,41465418
1,0.00,395.69,534.10,1306.86,417.34,0.000000,36.459130,38.086107,39.523066,16.828691,...,106.31,493,13,47,8,15,0,83,481.686995,88381735
2,0.00,0.00,0.00,9.95,16.00,0.000000,0.000000,0.000000,0.500000,0.000000,...,0.00,4,1,1,0,0,0,2,0.500000,89407327
3,43.80,1410.05,793.16,2662.01,448.47,0.000000,48.303099,18.106476,169.886459,21.064933,...,320.76,2615,26,118,21,80,2,247,754.840967,75367168
4,43.51,95.00,203.74,866.13,806.17,22.707125,0.000000,10.160607,85.839413,113.172128,...,444.16,490,32,47,9,4,2,94,828.539273,90262075


In [24]:
def create_last_df(df_train, df_test, df_musteri):
    
    df_train, df_test, df_musteri = df_train.copy(), df_test.copy(), df_musteri.copy()
    
    df_all = pd.concat([df_train, df_test])
    
    customer_identity_df = pd.merge(df_all, df_musteri, on="individualnumber", how="left")
    
    customer_identity_df["city_code"] = customer_identity_df["city_code"].fillna(82.0)
    customer_identity_df.loc[customer_identity_df["city_code"] > 81, "city_code"] = 82.0
    customer_identity_df["gender"] = customer_identity_df["gender"].fillna(customer_identity_df["gender"].mode()[0])
    
    sum_customer_df = pd.merge(customer_identity_df, sum_musteri_df, on="individualnumber", how="left")
    
    fill_first_df = sum_customer_df.groupby(["gender", "dateofbirth"]).transform(lambda x: x.fillna(x.mean()))
    filled_first_df = pd.concat([fill_first_df, sum_customer_df.loc[:, ["gender", "dateofbirth"]]], axis=1)
    
    sum_all_customer_df = pd.merge(filled_first_df, sum_musteri_categ_df, on=["individualnumber", "category_number"], how="left")
    sum_all_customer_df = sum_all_customer_df.drop(["category_number"], axis=1)
    sum_all_customer_df = sum_all_customer_df.set_index("individualnumber")
    
    fill_second_df = sum_all_customer_df.groupby(["gender", "dateofbirth"]).transform(lambda x: x.fillna(x.mean()))
    filled_second_df = pd.concat([fill_second_df, sum_all_customer_df.loc[:, ["gender", "dateofbirth"]]], axis=1)
    
    filled_second_df["city_code"] = filled_second_df["city_code"].astype(str)
    
    last_train_df = filled_second_df.loc[df_train["individualnumber"]]
    last_test_df = filled_second_df.loc[df_test["individualnumber"]]
    last_test_df = last_test_df.drop(["response"], axis=1)
    
    return last_train_df, last_test_df

In [25]:
my_last_train, my_last_test = create_last_df(train_df, test_df, customer_vector)
my_last_train.head()

,hakkedis_amt,odul_amt,response,city_code,is_sanal,amount,quantity,discount_type_1,discount_type_2,discount_type_3,amount_category,quantity_category,discount_type_1_category,discount_type_2_category,discount_type_3_category,is_sanal_category,gender,dateofbirth
individualnumber,,,,,,,,,,,,,,,,,,
94230288,21.0,2.0,0.0,7.0,1.0,4559.48,343.954,46.151577,170.85,209.63,202.51,36.0,10.595745,0.55,1.99,0.0,0.0,39.0
4684087,17.0,1.0,0.0,19.0,0.0,3383.80,152.397,99.388264,619.24,263.55,100.00,12.0,2.200982,0.70,0.00,0.0,0.0,71.0
92472145,24.0,3.0,0.0,35.0,7.0,14505.53,2108.974,1505.791730,695.24,1704.76,85.40,10.0,8.647374,0.00,0.00,2.0,1.0,40.0
88026681,22.0,2.0,0.0,82.0,38.0,6051.38,319.378,21.885276,39.15,244.70,78.05,8.0,0.000000,0.00,0.00,5.0,1.0,61.0
98127795,38.0,3.0,0.0,9.0,0.0,212.10,27.574,0.000000,0.00,1.20,38.00,2.0,0.000000,0.00,0.00,0.0,0.0,38.0


In [27]:
my_last_train = pd.merge(my_last_train.reset_index(), all_cust_df, on="individualnumber", how="left").drop_duplicates(subset="individualnumber").set_index("individualnumber")
my_last_train.head()

,hakkedis_amt,odul_amt,response,city_code,is_sanal_x,amount_x,quantity_x,discount_type_1_x,discount_type_2_x,discount_type_3_x,...,discount_type_2_y,discount_type_3_y,how_many_basket,diger_1,gida_1,hijyen_1,icecek_1,kisisel_bakim_1,sum_basket,sum_discount
individualnumber,,,,,,,,,,,,,,,,,,,,,
94230288,21.0,2.0,0.0,7.0,1.0,4559.48,343.954,46.151577,170.85,209.63,...,172.87,220.81,4041.0,168.0,181.0,7.0,36.0,1.0,393.0,441.452341
4684087,17.0,1.0,0.0,19.0,0.0,3383.80,152.397,99.388264,619.24,263.55,...,619.24,1298.71,990.0,38.0,67.0,13.0,20.0,0.0,138.0,2021.780001
92472145,24.0,3.0,0.0,35.0,7.0,14505.53,2108.974,1505.791730,695.24,1704.76,...,995.65,2549.79,21769.0,271.0,847.0,103.0,462.0,26.0,1709.0,5371.050969
88026681,22.0,2.0,0.0,82.0,38.0,6051.38,319.378,21.885276,39.15,244.70,...,39.15,271.04,2553.0,87.0,103.0,73.0,18.0,6.0,287.0,332.962566
98127795,38.0,3.0,0.0,9.0,0.0,212.10,27.574,0.000000,0.00,1.20,...,0.00,10.21,238.0,8.0,11.0,0.0,5.0,0.0,24.0,10.210000


In [28]:
my_last_test = pd.merge(my_last_test.reset_index(), all_cust_df, on="individualnumber", how="left").drop_duplicates(subset="individualnumber").set_index("individualnumber")
my_last_test.head()

,hakkedis_amt,odul_amt,city_code,is_sanal_x,amount_x,quantity_x,discount_type_1_x,discount_type_2_x,discount_type_3_x,amount_category,...,discount_type_2_y,discount_type_3_y,how_many_basket,diger_1,gida_1,hijyen_1,icecek_1,kisisel_bakim_1,sum_basket,sum_discount
individualnumber,,,,,,,,,,,,,,,,,,,,,
95812936,38.0,3.0,10.0,0.0,879.43,35.000,7.297619,0.00,18.34,108.359559,...,0.00,34.32,252.0,21.0,1.0,1.0,5.0,0.0,28.0,42.320000
23929570,28.0,2.0,34.0,0.0,845.16,55.922,15.459796,144.54,109.05,132.612122,...,149.54,109.05,159.0,9.0,20.0,4.0,9.0,3.0,45.0,278.414624
95948115,21.0,2.0,67.0,0.0,483.22,21.345,53.460776,11.20,15.26,114.290403,...,11.20,15.26,94.0,3.0,15.0,1.0,3.0,0.0,22.0,83.009999
100898513,48.0,4.0,51.0,0.0,3727.10,397.944,306.505670,114.20,129.88,310.600000,...,114.20,170.25,2859.0,72.0,125.0,10.0,48.0,2.0,257.0,662.630187
12769156,26.0,3.0,34.0,23.0,24372.04,3092.761,1021.535800,1248.55,805.21,54.800000,...,1336.19,1356.73,20684.0,398.0,777.0,119.0,272.0,8.0,1574.0,3837.954230


In [29]:
my_train_df = my_last_train.copy()
my_test_df = my_last_test.copy()